The history of twitter scraping is recorded on my github repo
>https://github.com/Frozha/twitterscraper

A brief explanation for the source code is provided henceforth.


### **IMPORTING RELEVANT MODULES**

In [ ]:
#importing selenium to stimulate web browser
#time is used to provide delays for page loading etc.
#bs4 is used to extract data from html format
#pandas and csv are used to store scraped data as a dataframe or csv
#os is used to access system environment variables
#pynput was used to log into twitter automatically

from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
import os
from selenium.webdriver.common.by import By
from pynput.keyboard import Key, Controller


### **SCRAPER FUNCTION DEFINATION**

In [ ]:
#function defination that returns the text of a div by providing the function with
#bs4 object, html_class in the tag, and the type of tag
def scraper(a, html_clss,tag):
    div_str = str(a.find(tag, class_=html_clss))
    div = bs(div_str,"html.parser")
    return div.text

### **AUTOMATIC TWITTER LOGIN**

```c
twitter login information was stored as environment variables on the workstation.
```

In [ ]:
keyboard = Controller()
url= "https://twitter.com/i/flow/login"
driver = webdriver.Firefox()
driver.get(url)
time.sleep(3)
keyboard.type(os.environ.get('TWT_USERN'))
keyboard.press(Key.enter)
time.sleep(2)
keyboard.type(os.environ.get('TWT_PSWD'))
driver.find_element(By.XPATH,'//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div').click()
time.sleep(3)


### **INITIALIZING DATAFRAME AND LOADING URLS**

In [ ]:
data = {
    'user_handle' : [],
    'tweet' : []
}
df1 = pd.DataFrame(data)

with open('url.txt','r') as file:
    urls = file.readlines()


### **SCRAPING TWEETS**

In [ ]:
for url in urls:
    driver.get(url)
    time.sleep(4)
    page_end = driver.execute_script("return document.body.scrollHeight;")
    time.sleep(5)

    tweet_div_class = "css-1dbjc4n r-eqz5dr r-16y2uox r-1wbh5a2"
    user_handle_class = "css-4rbku5 css-18t94o4 css-1dbjc4n r-1loqt21 r-1wbh5a2 r-dnmrzs r-1ny4l3l"
    daughter_tweet_class = "css-901oao r-1nao33i r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0"
    main_tweet_class = "css-901oao r-1nao33i r-37j5jr r-1inkyih r-16dba41 r-135wba7 r-bcqeeo r-bnwqim r-qvutc0"

    first_iteration = 1
    while True:
        driver.execute_script("window.scrollBy(0, 877);")
        time.sleep(3)
        current_page_end=driver.execute_script("return document.body.scrollHeight;")
        if current_page_end==page_end:
            break
        elif(first_iteration==1):
            '''taking first tweet that is the largest and has diff formatting before scrolling'''
            html = driver.page_source
            data['tweet']=[]
            data['user_handle']=[]
            soup = bs(html,"html.parser")
            for a in soup.find_all('div',class_=tweet_div_class):
                '''user handle logic'''
                data['user_handle'] = scraper(a,user_handle_class,'a')
                '''tweet logic'''
                if first_iteration==1:
                    data['tweet'] = scraper(a,main_tweet_class,'div')
                    first_iteration = 0
                else:
                    data['tweet'] = scraper(a,daughter_tweet_class,'div')
                '''writes to dataframe'''
                df1 = df1.append(data,ignore_index = True)

            '''after scrolling'''
        elif(first_iteration==0):
            html=driver.page_source
            soup = bs(html,"html.parser")
            data['tweet']=[]
            data['user_handle']=[]
            tempdf = pd.DataFrame(data)
            for a in soup.find_all('div',attrs=tweet_div_class):
                '''user handle logic'''
                data['user_handle'] = scraper(a,user_handle_class,'a')
                '''tweet logic'''
                data['tweet'] = scraper(a,daughter_tweet_class,'div')

                tempdf = tempdf.append(data,ignore_index = True)
            '''adds dataframes and removes duplicates'''
            df1=pd.concat([df1,tempdf]).drop_duplicates()
            del tempdf

        page_end = current_page_end

    print(df1)

             user_handle                                              tweet
0          Gaurav Sabnis  Modi's US visit this time has unleashed a wave...
1          Gaurav Sabnis  And it's all way more nuanced and hard hitting...
2          Gaurav Sabnis  Btw have you noticed that the right wing media...
3       Sharad Bandelkar  Wht is the accountability of Journalists?\nPol...
4         Godrej & Boyce  Prepare to be pleasantly surprised as we revea...
5                Prithvi  Yaayyyy....\n\nThe usual suspect....\n\nTheAtl...
6            Walrus Bird  From this piece, hard to read but one can’t he...
7   Srikanth Gonuguntla                                Best wishes to Mirza
8           Wakanda_1207  I certainly don't eat parathas and I love my m...
9                   Maan  It's worse than what you predicted a few weeks...
10          SANJAY HEGDE  Sorry Doc Singhvi. We south Indians have heard...
11          Grouchy Maxx  She is just as much a low-life as her master.\...
            

### **SAVING AS CSV AND CLOSING BROWSER**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df1.to_csv('/content/drive/MyDrive/Colab Notebooks/raw.csv')
driver.close()